### Import

In [1]:
import torch
import transformers
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer
import wikipedia
import nltk
#utile pour le pos tagging
#nltk.download('punkt')
#nltk.download('averaged_perceptron_tagger')

### Load Model

In [2]:
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

***
### Model function
Use loaded BERT model to return answer<br>
**Input** : Question, Context text <br>
**return** : Answer

In [3]:
def generate_answer(question, answer_text):
    print("I'm looking for an aswer, wait please ...")
    # == Tokenize ==
    # use a python dictonary so run on CPU
    # Apply the tokenizer to the input text, treating them as a text-pair.
    print("-Tokenization")
    input_ids = tokenizer.encode(question, answer_text)
    #print('The input has a total of {:} tokens.'.format(len(input_ids)))

    tokens = tokenizer.convert_ids_to_tokens(input_ids)
    # == Set Segment IDs ==
    # Search the input_ids for the first instance of the `[SEP]` token.
    sep_index = input_ids.index(tokenizer.sep_token_id)

    # The number of segment A tokens includes the [SEP] token istelf.
    num_seg_a = sep_index + 1

    # The remainder are segment B.
    num_seg_b = len(input_ids) - num_seg_a

    # Construct the list of 0s and 1s.
    segment_ids = [0]*num_seg_a + [1]*num_seg_b

    # There should be a segment_id for every input token.
    assert len(segment_ids) == len(input_ids)

    # == Run Model ==
    # Run our example through the model.
    # by default on CPU, use model.to(device) to select GPU !?
    print("-Forward pass on the model")
    start_scores, end_scores = model(torch.tensor([input_ids]), # The tokens representing our input text.
                                 token_type_ids=torch.tensor([segment_ids])) # The segment IDs to differentiate question from 
    # ici retourne un score pour chaque token
    
    # donc on applique un argmax pour trouver le plus probable
    # Find the tokens with the highest `start` and `end` scores.
    answer_start = torch.argmax(start_scores)
    answer_end = torch.argmax(end_scores)
    
    # Récupére le socre
    start_score = float(start_scores[0,answer_start])
    end_score = float(end_scores[0,answer_end])
    
    
    # == Print Answer without ## ==
    # Start with the first token.
    answer = tokens[answer_start]

    # Select the remaining answer tokens and join them with whitespace.
    for i in range(answer_start + 1, answer_end + 1):
    
        # If it's a subword token, then recombine it with the previous token.
        if tokens[i][0:2] == '##':
            answer += tokens[i][2:]
    
        # Otherwise, add a space then the token.
        else:
            answer += ' ' + tokens[i]

    return answer, start_score+end_score

    

In [4]:
generate_answer("What is IPO?", "Google was founded in 1998 by Larry Page and Sergey Brin while they were Ph.D. students at Stanford University in California. Together they own about 14 percent of its shares and control 56 percent of the stockholder voting power through supervoting stock. They incorporated Google as a privately held company on September 4, 1998. An initial public offering (IPO) took place on August 19, 2004, and Google moved to its headquarters in Mountain View, California, nicknamed the Googleplex. In August 2015, Google announced plans to reorganize its various interests as a conglomerate called Alphabet Inc. Google is Alphabet's leading subsidiary and will continue to be the umbrella company for Alphabet's Internet interests. Sundar Pichai was appointed CEO of Google, replacing Larry Page who became the CEO of Alphabet.")

I'm looking for an aswer, wait please ...
-Tokenization
-Forward pass on the model


('initial public offering', 14.472158908843994)

***
### Question Processing
Extract subjet _(and more?)_ from the question

In [5]:
def extract_subj(question):
    subject = None
    token = nltk.word_tokenize(question)
    #print(token)
    pos_token = nltk.pos_tag(token)
    for item in pos_token:
        if item[1] == 'NN':
            subject = item[0]
    if subject is not None:
        print("Subject found: " + subject)
    else:
        print("Subject not found 😔\n Rephrase the question or try another one")
    return  subject



In [8]:
import spacy
import en_core_web_sm

import truecase

nlp = en_core_web_sm.load()
#!python -m spacy download en_core_web_sm
#nlp = spacy.load("en_core_web_sm")

def extract_subject(question):
    #noun to not take in count
    subj_list = []
    osef_list = ['who','why','what','when','which','how']
    doc = nlp(question)
    
    ents = doc.ents
    nouns = doc.noun_chunks
    # == Check for Named Entity ==
    if len(ents) != 0:
        for ent in doc.ents:
            subj_list.append(ent.text)
            print("Named entity: " + ent.text) #, ent.start_char, ent.end_char, ent.label_
    # == Check for Noun Chunk ==
    elif len(list(nouns)) != 0:
        for item in nouns:
            if str(item) not in osef_list:
                subj_list.append(str(item))
                print("Noun chunck: " + str(item))
                
    #print("Subject found: " + str(item))
    #return str(item)
    #print("Subject not found 😔\n Rephrase the question or try another one")
    if len(subj_list) != 0:
        print("Subject selected: " + subj_list[-1])
        return subj_list[-1]
    else:
        print("Subject not found 😔\n Rephrase the question or try another one")
        return None
#test
#extract_subject_with_spacy('which is the most common use of opt-in e-mail marketing ?')

In [12]:
question = "how darth vader dies?"

question = truecase.get_true_case(question)
print(question)

extract_subject(question)

How Darth Vader dies?
<class 'tuple'>
<class 'generator'>
Named entity: Darth Vader
Subject selected: Darth Vader


'Darth Vader'

In [35]:
import spacy

nlp = spacy.load("en_core_web_sm")
doc = nlp("In")
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)
    

True


### Wikipedia API
Try to found the most relevant context text to give as BERT input. <br>
Get a wikipedia article, and scrap it

In [12]:
# == WIP ==
# découpé en paragraphe (le model a une limite de 512 token pour le text en entré)
def get_wiki_and_split(subject):
    text = wikipedia.summary(subject)
    print(len(text))
    return text


In [19]:
wikipedia.search("star wars mace windu")

['Mace Windu',
 'Star Wars: The Clone Wars (2002 video game)',
 'Star Wars: Clone Wars (2003 TV series)',
 'Star Wars: The Rise of Skywalker',
 'Terrence C. Carson',
 'List of Star Wars: The Clone Wars cast members',
 'Star Wars: The Clone Wars (2008 TV series)',
 'Jango Fett',
 'Star Wars: Episode III – Revenge of the Sith',
 'Star Wars: Episode II – Attack of the Clones']

In [14]:
text = wikipedia.summary('salsa')
list = text.split('.')
print(list)

['Samba (Portuguese pronunciation: [ˈsɐ̃bɐ] (listen)), also known as samba urbano carioca (Urban Carioca Samba) or simply samba carioca (Carioca Samba) is a Brazilian music genre that originated in the Afro-Brazilians communities of Rio de Janeiro in the early 20th century', ' Having its roots in the cultural expression of West Africa and in Brazilian folk traditions, especially those linked to the primitive rural samba of the colonial and imperial periods, is considered one of the most important cultural phenomena in Brazil and one of the country symbols Present in the Portuguese language at least since the 19th century, the word “samba” was originally used to designate a “popular dance”', ' Over time, its meaning has been extended to a “batuque-like circle dance”, a dance style and also to a “music genre”', ' This process of establishing itself as a musical genre began in the 1910s and it had its inaugural landmark in the song “Pelo Telefone”, launched in 1917', ' Despite being ident

***
### Visualization

In [17]:
from IPython.display import display
from ipywidgets import widgets

#Widgets layout difinition
layout = widgets.Layout(width='400px', height='0px', margin='100px 0 0 100px')
bLayout = widgets.Layout(width='50px', height='28px', margin='100px 0 0 0px')
outLayoutPropre = widgets.Layout(width='480px', height='100px', margin='50px 0 100px 100px')
outLayoutTest = widgets.Layout(width='450px', height='auto', margin='50px 0 100px 100px')
#titleLayout = widgets.Layout(width='450px', height='auto', margin='0px 0 0px 100px')
 
#Widgets object definition
text = widgets.Text(layout=layout)
button = widgets.Button(description = 'Ask', layout = bLayout)
out = widgets.Output(layout=outLayoutTest)#layout=outLayout
#out = widgets.HTML(layout = outLayout, value= '<style>.text {width: 480px; heigh: 100px;}</style> <p class="text">'+ out_value +' </p>')
 
def button_on_click(self):
    with out:
        out.clear_output()
        
        subject = extract_subject(question=text.value)
        
        if subject is not None:
            
            context = wikipedia.summary(subject)
            
            answer, score = generate_answer(text.value, context[:2000])
            #out.clear_output()
            print()
            print("Here what i found: \n"+ answer)
            print()
            print("Score: " +str(score))
        else:
            pass
        
button.on_click(button_on_click)

display(widgets.HBox((text, button,)))
display(out)

Output(layout=Layout(height='auto', margin='50px 0 100px 100px', width='450px'))